In [1]:
!pip install transformers
!pip install torch
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!python -c "import torch; print(torch.__version__)"

2.5.1+cu124


In [1]:
# imports

import json
import pandas as pd


from collections import defaultdict

In [3]:
user_history_filepath = "/scratch/general/vast/u1471428/hugging_face_cache/user_history_full_data.json" 
frequent_bought_filepath = "/scratch/general/vast/u1471428/hugging_face_cache/frequent_pairs_data.json"
product_dict_filepath = "/scratch/general/vast/u1471428/hugging_face_cache/product_dictionary.json" 

In [4]:
json_file = open(user_history_filepath, 'r')
user_history = json.load(json_file)
json_file.close()

f_json_file = open(frequent_bought_filepath, 'r')
frequent_bought = json.load(f_json_file)
f_json_file.close()

p_json_file = open(product_dict_filepath, 'r')
product_dictionary = json.load(p_json_file)
p_json_file.close()

In [5]:

def parse_frequent_bought(frequent_bought):
    """
    Parse the frequent bought pairs into a product-to-products dictionary.
    Args:
        frequent_bought (dict): Original frequent bought pairs with frequencies.
    Returns:
        product_to_frequent (dict): Dictionary mapping each product ID to a set of frequently bought products.
    """
    product_to_frequent = defaultdict(set)

    for pair, frequency in frequent_bought.items():
        # Split the product pair into individual product IDs
        product1, product2 = pair.split(',')
        # Add each product to the other's set
        product_to_frequent[product1].add(product2)
        product_to_frequent[product2].add(product1)

    return product_to_frequent

product_to_frequent = parse_frequent_bought(frequent_bought)
print(product_to_frequent)


defaultdict(<class 'set'>, {'B0BQV3785S': {'B0C3WX29HS', 'B07C2HRMRF', 'B0BFHP8JPG', 'B0BWS6X4WF', 'B0C68XP6K8', 'B09YYZTK1W', 'B07XM9DX9H', 'B0BMQ3G124', 'B09KLYL2ZX', 'B0C54J5D2B', 'B08K41XMBJ', 'B0B63QBWFQ'}, 'B0C54J5D2B': {'B0BQV3785S', 'B0BFHP8JPG', 'B0C3F1KFPR', 'B09YYZTK1W', 'B0BY8YWJWG', 'B0BMQ3G124', 'B09KLYL2ZX', 'B0C3WX29HS', 'B0BX6HYYQ2'}, 'B09KLYL2ZX': {'B0BQV3785S', 'B07C2HRMRF', 'B0C614K38T', 'B0C3F1KFPR', 'B09YYZTK1W', 'B07XM9DX9H', 'B0BMQ3G124', 'B0BXX7LB93', 'B0C54J5D2B', 'B0C3WX29HS', 'B0B63QBWFQ'}, 'B0C68XP6K8': {'B0C3F1KFPR', 'B0BQV3785S'}, 'B0BKV9ZGLH': {'B0CDX5J8C6', 'B09K4SCPYT', 'B0C59SYPNT', 'B0BWS6X4WF', 'B0C6MZ7GVW', 'B07TSG87YD', 'B07XM9DX9H', 'B00OQCZAVW', 'B00ECHYTBI', 'B0C3RDQCL4'}, 'B0C59SYPNT': {'B0BB84JXS9', 'B084SZ4BQB', 'B09K4SCPYT', 'B07L5FC123', 'B0B953DKMY', 'B0C9V7CB9S', 'B0C7XCXYHC', 'B0C75PQQXN', 'B0BN714TVL', 'B07XM9DX9H', 'B07SP24DL5', 'B00OQCZAVW', 'B000BNCA4K', 'B09S8PT9L6', 'B0BKV9ZGLH', 'B0C3RDQCL4', 'B006VB29D8'}, 'B09YYZTK1W': {'B0BQV3

In [6]:
def count_user_with_history_sizes(user_h):
    cnt_dict = defaultdict(int)
    for user_id in user_history.keys():
        cnt_dict[len(user_history[user_id])]+=1
    print(cnt_dict)
    
def count_user_with_history_size_above(user_h, above):
    cnt = 0
    for user_id in user_history.keys():
        if len(user_history[user_id]) >= above:
            cnt+=1
    print(cnt)

def filter_users(user_h, min_history_length, max_history_length=-1)->dict:
    filtered_users = {}
    count=0
    for user, history in user_h.items():
        if len(history) >= min_history_length:
            filtered_users[user] = history[:max_history_length]
            count+=1
        
        if count==-1:
            break
    return filtered_users
        
    

In [7]:
filtered_users = filter_users(user_history, 20, 50)
print(len(filtered_users))

7297


In [8]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from collections import Counter

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [10]:
users = list(filtered_users.keys())

# Split into train (80%) and test (20%)
train_users, test_users = train_test_split(users, test_size=0.2, random_state=42)

# Further split train users into train (80%) and validation (20%) within the training set
train_users, val_users = train_test_split(train_users, test_size=0.2, random_state=42)

# Create train, validation, and test data dictionaries
train_data = {user: filtered_users[user] for user in train_users}
val_data = {user: filtered_users[user] for user in val_users}
test_data = {user: filtered_users[user] for user in test_users}

# Print dataset sizes for verification
print(f"Train users: {len(train_users)}, Validation users: {len(val_users)}, Test users: {len(test_users)}")

Train users: 4669, Validation users: 1168, Test users: 1460


In [11]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
main_category_encoder = LabelEncoder()
category_encoder = LabelEncoder()
product_encoder = LabelEncoder()

In [12]:
main_categories = [entry["main_category"] for user in user_history.keys() for entry in user_history[user]]
main_category_encoder.fit(main_categories)
categories = [entry["categories"] for user in user_history.keys() for entry in user_history[user]]
category_encoder.fit(categories)
product_ids = [entry["product_id"] for user in user_history.keys() for entry in user_history[user]]
product_encoder.fit(product_ids)

LabelEncoder()

In [13]:
def normalize_ratings(histories):
    all_ratings = [h["rating"] for user in histories for h in histories[user]]
    scaler = MinMaxScaler(feature_range=(0,1))
    scaler.fit([[r] for r in all_ratings])
    return scaler

rating_scaler = normalize_ratings(filtered_users)

def preprocess_history(history):
    texts = [
        f"{h.get('review_title','')} {h.get('features','')} {h.get('main_category')}" for h in history
    ]
    
    texts = [text for text in texts if text.strip()]
    
    tokens = tokenizer(texts, padding="max_length", truncation=True, return_tensors="pt", max_length=128)
    
    ratings = torch.tensor([rating_scaler.transform([[h["rating"]]])[0][0] for h in history], dtype=torch.float32)
    
    #categories = torch.tensor(category_encoder.transform([h["main_category"] for h in history]))
    
    categories = torch.tensor(category_encoder.transform([h["categories"] for h in history]))
    
    product_ids = torch.tensor(product_encoder.transform([h["product_id"] for h in history]))
    
    return tokens, ratings, categories, product_ids

In [14]:
class UserDataset(Dataset):
    def __init__(self, data, tokenizer, category_encoder, product_encoder, product_to_frequent, is_train=False, seq_len=15, pred_len=5):
        self.data = data
        self.tokenizer = tokenizer
        self.category_encoder = category_encoder # Is it required?
        self.product_encoder = product_encoder # Is it required?
        self.product_to_frequent = product_to_frequent
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.is_train = is_train
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        user, history = list(self.data.items())[idx]
        tokens, ratings, categories, product_ids = preprocess_history(history)
        input_tokens = {
            "input_ids":tokens["input_ids"][:self.seq_len],
            "attention_mask": tokens["attention_mask"][:self.seq_len],
            "token_type_ids": tokens["token_type_ids"][:self.seq_len],
        }
        input_ratings = ratings[:self.seq_len]
        input_categories = categories[:self.seq_len]
        
        future_products = product_ids[self.seq_len:]
        target_vector = torch.zeros(len(self.product_encoder.classes_))
        target_vector[future_products] = 1
        
        if self.is_train:
            for product_id in product_ids[:self.seq_len]:
                frequent_set = self.product_to_frequent.get(self.product_encoder.classes_[product_id.item()], set())
#                 print(frequent_set)
                encoded_frequent_products = self.product_encoder.transform(list(frequent_set))
                target_vector[encoded_frequent_products]=1
                
        
        return input_tokens, input_ratings, input_categories, target_vector

train_dataset = UserDataset(train_data, tokenizer, category_encoder, product_encoder, product_to_frequent, is_train=True)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [15]:
class SingleVectorTransformerRecommendationModel(nn.Module):
    def __init__(self, bert_model_name="bert-base-uncased", num_categories=900, num_products=1000, d_model=128, nhead=8, num_encoder_layers=3):
        super(SingleVectorTransformerRecommendationModel, self).__init__()
        
        self.bert = BertModel.from_pretrained(bert_model_name)
        bert_hidden_size = self.bert.config.hidden_size
        
        self.category_embedding = nn.Embedding(num_categories, d_model)
        
        self.history_encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.history_encoder = nn.TransformerEncoder(self.history_encoder_layer, num_layers = num_encoder_layers)
        
        self.input_projection = nn.Linear(bert_hidden_size + d_model, d_model)
        
        self.fc_out = nn.Linear(d_model, num_products)
        self.sigmoid = nn.Sigmoid()
        
        self.layer_norm = nn.LayerNorm(d_model)
        self.activation = nn.GELU()
        self.init_weights()
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, tokens, ratings, categories):
        batch_size, seq_len, max_token_length = tokens["input_ids"].shape
        
        input_ids = tokens["input_ids"].view(-1, max_token_length)
        attention_mask = tokens["attention_mask"].view(-1, max_token_length)
        token_type_ids = tokens["token_type_ids"].view(-1, max_token_length)
        
        bert_output = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids
        )
        sequence_output = bert_output.last_hidden_state
        
        sequence_output = sequence_output[:,0,:].view(batch_size, seq_len, -1)
        
        category_embeds = self.category_embedding(categories)
        combined_features = torch.cat([sequence_output, category_embeds], dim=-1)
        
        projected_features = self.input_projection(combined_features)  # Shape: [batch_size, seq_len, d_model]
        normalized_features = self.layer_norm(projected_features)
        activated_features = self.activation(normalized_features) 

        # Encode history with Transformer
        history_encoded = self.history_encoder(activated_features)
        
        
#         history_encoded = self.history_encoder(combined_features)
        aggregated_features = history_encoded.mean(dim=1)
        aggregated_features = self.layer_norm(aggregated_features)
        
        
        logits = self.fc_out(aggregated_features)
        probabilities = self.sigmoid(logits)
        
        return probabilities
        

In [16]:
def compute_top_k_accuracy(predicted_vector, target_vector, k=5):
    """
    Compute Top-K accuracy for multi-label classification.
    Args:
        predicted_vector (Tensor): Predicted probabilities for products [batch_size, num_products].
        target_vector (Tensor): Binary target vector [batch_size, num_products].
        k (int): Number of top predictions to consider.
    Returns:
        top_k_accuracy (float): Top-K accuracy for the batch.
    """
    # Get indices of the top-k predictions for each batch
    top_k_preds = torch.topk(predicted_vector, k=k, dim=-1).indices  # [batch_size, k]

    # Gather the target values corresponding to the top-k predictions
    true_positives = target_vector.gather(1, top_k_preds)  # [batch_size, k]

    # Count how many of the top-k predictions are correct
    top_k_correct = true_positives.sum(dim=-1)  # [batch_size]

    # Compute the accuracy as the mean of correct predictions
    top_k_accuracy = (top_k_correct > 0).float().mean().item()

    return top_k_accuracy

In [32]:
def compute_top_k_metrics(predicted_vector, target_vector, k=5):
    """
    Compute top-k accuracy and precision for classification tasks.
    
    Args:
        predicted_vector (torch.Tensor): Predicted probabilities or logits
        target_vector (torch.Tensor): Ground truth labels
        k (int, optional): Number of top predictions to consider. Defaults to 5.
    
    Returns:
        dict: A dictionary containing top-k accuracy and precision
    """
    # Get indices of the top-k predictions for each batch 
    top_k_preds = torch.topk(predicted_vector, k=k, dim=-1).indices  # [batch_size, k] 
 
    # Gather the target values corresponding to the top-k predictions 
    true_positives = target_vector.gather(1, top_k_preds)  # [batch_size, k] 
 
    # Compute top-k accuracy
    top_k_correct = true_positives.sum(dim=-1)  # [batch_size] 
    top_k_accuracy = (top_k_correct > 0).float().mean().item()
    
    # Compute top-k precision
    # Precision = (number of correct predictions in top-k) / (total number of top-k predictions)
    correct_predictions_count = true_positives.sum()
    top_k_precision = correct_predictions_count / (top_k_preds.shape[0] * k)
    
    return top_k_accuracy,top_k_precision.item()

In [36]:


def evaluate_top_k_accuracy(model, test_loader, k=5):
    """
    Evaluate average Top-K accuracy over the test set with a progress bar.
    Args:
        model (nn.Module): Trained model to evaluate.
        test_loader (DataLoader): DataLoader for the test dataset.
        k (int): Number of top predictions to consider.
    Returns:
        average_top_k_accuracy (float): Average Top-K accuracy over all test batches.
    """
    model.eval()
    total_top_k_accuracy = 0.0
    total_top_k_precision = 0.0
    
    total_batches = 0

    # Add a progress bar
    progress_bar = tqdm(test_loader, desc="Evaluating", leave=True)

    with torch.no_grad():
        for batch_tokens, ratings, categories, target_vector in progress_bar:
            # Move inputs and targets to the device
            tokens = {key: val.to(device) for key, val in batch_tokens.items()}
            ratings = ratings.to(device)
            categories = categories.to(device)
            target_vector = target_vector.to(device)

            # Make predictions
            predicted_vector = model(tokens, ratings, categories)

            # Compute Top-K accuracy for the batch
            top_k_acc, top_k_prec= compute_top_k_metrics(predicted_vector, target_vector, k=k)

            # Update total accuracy and batch count
            total_top_k_accuracy += top_k_acc
            total_top_k_precision += top_k_prec
            total_batches += 1

            # Update progress bar
            progress_bar.set_postfix({"Batch Top-K Acc": top_k_acc,
                                     "Batch Top-K prec": top_k_prec})

    # Compute average Top-K accuracy
    average_top_k_accuracy = total_top_k_accuracy / total_batches
    print(f"Average Top-{k} Accuracy: {average_top_k_accuracy:.4f}")
    average_top_k_precision = total_top_k_precision / total_batches
    print(f"Average Top-{k} Precision: {average_top_k_precision:.4f}")
    return average_top_k_accuracy

# Example usage:
# average_top_k = evaluate_top_k_accuracy(model, test_loader, k=5)
val_dataset = UserDataset(val_data, tokenizer, category_encoder, product_encoder, product_to_frequent)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=True)




In [18]:
model = SingleVectorTransformerRecommendationModel(num_categories=len(category_encoder.classes_), num_products=len(product_encoder.classes_)).to(device)
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [20]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}", leave=True)

    for tokens, ratings, categories, target_vector in progress_bar:
        tokens = {key: val.to(device) for key, val in tokens.items()}
        ratings = ratings.to(device)
        categories = categories.to(device)
        target_vector = target_vector.to(device)

        optimizer.zero_grad()

        # Forward pass
        predicted_vector = model(tokens, ratings, categories)

        # Compute loss
        loss = loss_fn(predicted_vector, target_vector.float())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Update progress bar
        progress_bar.set_postfix({"Loss": loss.item()})
    
    average_top_k = evaluate_top_k_accuracy(model, val_loader, k=5)
    print(f"Epoch {epoch+1} Val result top k acc {average_top_k}")

    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {total_loss / len(train_loader):.4f}")


Evaluating: 100%|██████████| 292/292 [03:20<00:00,  1.46it/s, Batch Top-K Acc=0]   


Average Top-5 Accuracy: 0.1122
Epoch 1 Val result top k acc 0.11215753424657535
Epoch 1/3, Average Loss: 0.7005


Evaluating: 100%|██████████| 292/292 [03:20<00:00,  1.46it/s, Batch Top-K Acc=0]   


Average Top-5 Accuracy: 0.1122
Epoch 2 Val result top k acc 0.11215753424657535
Epoch 2/3, Average Loss: 0.6932


Epoch 3:   0%|          | 1/584 [00:06<1:00:36,  6.24s/it, Loss=0.693]


KeyboardInterrupt: 

In [21]:
torch.save(model.state_dict(), "single_vector_data_augmented.pth")
print("Model's state dictionary saved to 'single_vector_data_augmented.pth'")

Model's state dictionary saved to 'single_vector_data_augmented.pth'


In [19]:
model.load_state_dict(torch.load("single_vector_data_augmented.pth"))

/scratch/local/u1471428/2606234/ipykernel_201366/1839281486.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("single_vector_data_augmente

<All keys matched successfully>

In [34]:
test_dataset = UserDataset(test_data, tokenizer, category_encoder, product_encoder, product_to_frequent)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [37]:
average_top_k = evaluate_top_k_accuracy(model, test_loader, k=5)
# print(f"Epoch {epoch+1} Val result top k acc {average_top_k}")
average_top_k = evaluate_top_k_accuracy(model, test_loader, k=3)
# print(f"Epoch {epoch+1} Val result top k acc {average_top_k}")
average_top_k = evaluate_top_k_accuracy(model, test_loader, k=1)
# print(f"Epoch {epoch+1} Val result top k acc {average_top_k}")

Evaluating: 100%|██████████| 1460/1460 [04:16<00:00,  5.69it/s, Batch Top-K Acc=0, Batch Top-K prec=0]  


Average Top-5 Accuracy: 0.1185
Average Top-5 Precision: 0.0263


Evaluating: 100%|██████████| 1460/1460 [04:16<00:00,  5.69it/s, Batch Top-K Acc=0, Batch Top-K prec=0]    


Average Top-3 Accuracy: 0.0781
Average Top-3 Precision: 0.0276


Evaluating: 100%|██████████| 1460/1460 [04:16<00:00,  5.69it/s, Batch Top-K Acc=0, Batch Top-K prec=0]

Average Top-1 Accuracy: 0.0397
Average Top-1 Precision: 0.0397


In [55]:


def ana_top_k_accuracy(model, test_loader, k=5):
    """
    Evaluate average Top-K accuracy over the test set with a progress bar.
    Args:
        model (nn.Module): Trained model to evaluate.
        test_loader (DataLoader): DataLoader for the test dataset.
        k (int): Number of top predictions to consider.
    Returns:
        average_top_k_accuracy (float): Average Top-K accuracy over all test batches.
    """
    model.eval()
    total_top_k_accuracy = 0.0
    total_batches = 0

    # Add a progress bar
    progress_bar = tqdm(test_loader, desc="Evaluating", leave=True)

    with torch.no_grad():
        for batch_tokens, ratings, categories, target_vector in progress_bar:
            # Move inputs and targets to the device
            tokens = {key: val.to(device) for key, val in batch_tokens.items()}
            ratings = ratings.to(device)
            categories = categories.to(device)
            target_vector = target_vector.to(device)

            # Make predictions
            predicted_vector = model(tokens, ratings, categories)

            # Compute Top-K accuracy for the batch
            top_k_acc, top_k_prec = compute_top_k_metrics(predicted_vector, target_vector, k=k)
            
            top_k_preds = torch.topk(predicted_vector, k=k, dim=-1).indices  # [batch_size, k]

            # Gather the target values corresponding to the top-k predictions
            true_positives = target_vector.gather(1, top_k_preds)  # [batch_size, k]

            # Count how many of the top-k predictions are correct
            top_k_correct = true_positives.sum(dim=-1)  # [batch_size]

            # Compute the accuracy as the mean of correct predictions
            top_k_accuracy = (top_k_correct > 0).float().mean().item()
            
            if top_k_accuracy>=0.5:
                print(top_k_correct.float().item())
                to_print = analyze_predictions_and_targets(top_k_preds, target_vector)
                print(json.dumps(to_print, indent=2))
                break
                
            # Update total accuracy and batch count
            total_top_k_accuracy += top_k_acc
            total_batches += 1

            # Update progress bar
            progress_bar.set_postfix({"Batch Top-K Acc": top_k_acc})

    # Compute average Top-K accuracy
#     average_top_k_accuracy = total_top_k_accuracy / total_batches
#     print(f"Average Top-{k} Accuracy: {average_top_k_accuracy:.4f}")
#     return average_top_k_accuracy


In [56]:
import json

def analyze_predictions_and_targets(top_k_preds, target_vector):
    """
    Analyze predictions to determine which items are in predictions but not in targets,
    which are in both predictions and targets, and which are in the target vector.

    Args:
        top_k_preds (Tensor): Indices of top-K predicted products [batch_size, k].
        target_vector (Tensor): Binary target vector for products [batch_size, num_products].
        product_encoder (LabelEncoder): Encoder to map product indices to product IDs.
        product_dictionary (dict): Dictionary containing product details (title, rating, etc.).

    Returns:
        result (list of dict): Contains information about items in `top_k_preds` but not in `target_vector`,
                               items in both `top_k_preds` and `target_vector`,
                               and items in `target_vector`.
    """
    result = []
    
    # Convert product indices to product IDs
    product_indices = torch.arange(target_vector.size(-1))  # [num_products]
    product_ids = product_encoder.inverse_transform(product_indices.cpu().numpy())  # Decode all product IDs
    
    for batch_idx in range(top_k_preds.size(0)):
        preds = top_k_preds[batch_idx].cpu().numpy()  # Predicted indices
        targets = torch.where(target_vector[batch_idx] > 0)[0].cpu().numpy()  # Target indices
        
        preds_product_ids = [product_ids[i] for i in preds]  # Get product IDs for predictions
        targets_product_ids = [product_ids[i] for i in targets]  # Get product IDs for targets
        
        # Products in predictions but not in targets
        preds_not_in_targets = list(set(preds_product_ids) - set(targets_product_ids))
        
        # Products in both predictions and targets
        preds_in_targets = list(set(preds_product_ids) & set(targets_product_ids))
        
        # Products only in the target vector
        target_only = list(set(targets_product_ids))
        
        # Fetch details from the product dictionary
        items_not_in_targets = [
            {
                "product_id": pid,
                "title": product_dictionary.get(pid, {}).get("title", "Unknown"),
                "rating": product_dictionary.get(pid, {}).get("average_rating", "Unknown"),
            }
            for pid in preds_not_in_targets
        ]
        
        items_in_targets = [
            {
                "product_id": pid,
                "title": product_dictionary.get(pid, {}).get("title", "Unknown"),
                "rating": product_dictionary.get(pid, {}).get("average_rating", "Unknown"),
            }
            for pid in preds_in_targets
        ]
        
        items_in_target_vector = [
            {
                "product_id": pid,
                "title": product_dictionary.get(pid, {}).get("title", "Unknown"),
                "rating": product_dictionary.get(pid, {}).get("average_rating", "Unknown"),
            }
            for pid in target_only
        ]
        
        result.append(
            {
                "batch_index": batch_idx,
                "not_in_targets": items_not_in_targets,
                "in_targets": items_in_targets,
                "in_target_vector": items_in_target_vector,
            }
        )
    
    return result

def print_results_as_json(results):
    """
    Print results in JSON format with indent=2.
    Args:
        results (list of dict): The result of analyze_predictions_and_targets.
    """
    print(json.dumps(results, indent=2))


In [57]:
test_dataset = UserDataset(test_data, tokenizer, category_encoder, product_encoder, product_to_frequent)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [41]:
ana_top_k_accuracy(model, test_loader, k=5)

Evaluating:   0%|          | 2/1460 [00:00<05:50,  4.16it/s, Batch Top-K Acc=1]

[
  {
    "batch_index": 0,
    "not_in_targets": [
      {
        "product_id": "B07TSG87YD",
        "title": "HUGGIES Snug & Dry Diapers, Size 5, 96 Count, GIGA JR PACK (Packaging May Vary)",
        "rating": 4.6
      },
      {
        "product_id": "B07XM9DX9H",
        "title": "Diapers Newborn/Size 1 (8-14 lb), 84 Count - Pampers Swaddlers Sensitive Disposable Baby Diapers, Super Pack",
        "rating": 4.8
      },
      {
        "product_id": "B00OQCZAVW",
        "title": "Baby Banana Yellow Banana Infant Toothbrush, Easy to Hold, Made in the USA, Train Infants Babies and Toddlers for Oral Hygiene, Teether Effect for Sore Gums, 4.33\" x 0.39\" x 7.87\", BR003",
        "rating": 4.8
      },
      {
        "product_id": "B00ECHYTBI",
        "title": "Infant Optics DXR-8 Video Baby Monitor, Non-WiFi Hack-Proof FHSS Connection, Interchangeable Lenses, Pan Tilt Zoom, LED Sound Bar, Night Vision, and Two-way Talk",
        "rating": 4.5
      },
      {
        "product_id

In [58]:
ana_top_k_accuracy(model, test_loader, k=5)

Evaluating:   1%|          | 16/1460 [00:03<04:41,  5.12it/s, Batch Top-K Acc=0]

1.0
[
  {
    "batch_index": 0,
    "not_in_targets": [
      {
        "product_id": "B00OQCZAVW",
        "title": "Baby Banana Yellow Banana Infant Toothbrush, Easy to Hold, Made in the USA, Train Infants Babies and Toddlers for Oral Hygiene, Teether Effect for Sore Gums, 4.33\" x 0.39\" x 7.87\", BR003",
        "rating": 4.8
      },
      {
        "product_id": "B00ECHYTBI",
        "title": "Infant Optics DXR-8 Video Baby Monitor, Non-WiFi Hack-Proof FHSS Connection, Interchangeable Lenses, Pan Tilt Zoom, LED Sound Bar, Night Vision, and Two-way Talk",
        "rating": 4.5
      },
      {
        "product_id": "B07XM9DX9H",
        "title": "Diapers Newborn/Size 1 (8-14 lb), 84 Count - Pampers Swaddlers Sensitive Disposable Baby Diapers, Super Pack",
        "rating": 4.8
      },
      {
        "product_id": "B07TSG87YD",
        "title": "HUGGIES Snug & Dry Diapers, Size 5, 96 Count, GIGA JR PACK (Packaging May Vary)",
        "rating": 4.6
      }
    ],
    "in_targets":